In [1]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.start = min(coord1, coord2)
        self.end = max(coord1, coord2)
        self.supports = set()
        self.supported_by = set()

    def occupies(self, coord):
        return all(self.start[i] <= coord[i] <= self.end[i] for i in range(3))

    def __repr__(self):
        return f"Brick{id}: {self.start}~{self.end}"


class World:
    def __init__(self):
        self.bricks = []
        self.brick_map = {}  # Mapping of coordinates to bricks

    def add_brick(self, brick):
        self.bricks.append(brick)
        for x in range(brick.start[0], brick.end[0] + 1):
            for y in range(brick.start[1], brick.end[1] + 1):
                for z in range(brick.start[2], brick.end[2] + 1):
                    self.brick_map[(x, y, z)] = brick

    def settle_bricks(self):
        for brick in self.bricks:
            # Check below each brick to find supports
            for x in range(brick.start[0], brick.end[0] + 1):
                for y in range(brick.start[1], brick.end[1] + 1):
                    below = (x, y, brick.start[2] - 1)
                    if below in self.brick_map:
                        supporting_brick = self.brick_map[below]
                        brick.supported_by.add(supporting_brick)
                        supporting_brick.supports.add(brick)

    def simulate_disintegration(self):
        safe_disintegrations = 0
        for brick in self.bricks:
            # A brick can be disintegrated if it's not supporting any bricks,
            # or if all bricks it's supporting are also supported by other bricks
            if not brick.supports or all(any(other_brick in support.supported_by for other_brick in brick.supports if other_brick != support) for support in brick.supports):
                safe_disintegrations += 1
        return safe_disintegrations


def parse_input(input_lines):
    world = World()
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        start = tuple(map(int, coords[0].split(',')))
        end = tuple(map(int, coords[1].split(',')))
        brick = Brick(id, start, end)
        world.add_brick(brick)
    return world


# Main Function
def main(input_lines):
    world = parse_input(input_lines)
    world.settle_bricks()
    return world.simulate_disintegration()


# Read the input file
input_file_path = "input.txt"  # Ensure this path is correct for your environment

with open(input_file_path, "r") as file:
    input_lines = file.readlines()

# Running the function with the input file content and getting the result
safe_disintegrations = main(input_lines)
print(f"Number of bricks that can be safely disintegrated: {safe_disintegrations}")


Number of bricks that can be safely disintegrated: 863


In [2]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.start = tuple(min(a, b) for a, b in zip(coord1, coord2))
        self.end = tuple(max(a, b) for a, b in zip(coord1, coord2))
        self.supports = set()

    def is_supported(self, brick_map):
        # Check if the brick is on the ground or if there's another brick directly beneath any part of it
        if self.start[2] == 1:  # On the ground
            return True
        for x in range(self.start[0], self.end[0] + 1):
            for y in range(self.start[1], self.end[1] + 1):
                # Check the space directly below this part of the brick
                if (x, y, self.start[2] - 1) in brick_map:
                    return True
        return False

    def __repr__(self):
        return f"Brick{id}: {self.start}~{self.end}"


class World:
    def __init__(self):
        self.bricks = []
        self.brick_map = {}  # Mapping of coordinates to bricks

    def add_brick(self, brick):
        self.bricks.append(brick)
        # Add all coordinates occupied by this brick to the brick map
        for x in range(brick.start[0], brick.end[0] + 1):
            for y in range(brick.start[1], brick.end[1] + 1):
                for z in range(brick.start[2], brick.end[2] + 1):
                    self.brick_map[(x, y, z)] = brick

    def settle_bricks(self):
        # Sort bricks by their lowest z-coordinate so that we process lower bricks first
        for brick in sorted(self.bricks, key=lambda b: b.start[2]):
            if not brick.is_supported(self.brick_map):
                raise ValueError(f"Brick {brick.id} is not stable!")

    def simulate_disintegration(self):
        safe_disintegrations = 0
        for brick in self.bricks:
            # A brick can be disintegrated safely if removing it doesn't make any other brick unsupported
            original_supports = set(brick.supports)
            for supported in original_supports:
                supported.supports.remove(brick)  # Simulate removing this brick
            # Check if any brick is unsupported now
            if all(supported.is_supported(self.brick_map) for supported in original_supports):
                safe_disintegrations += 1
            # Revert changes
            for supported in original_supports:
                supported.supports.add(brick)
        return safe_disintegrations


def parse_input(input_lines):
    world = World()
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        start = tuple(map(int, coords[0].split(',')))
        end = tuple(map(int, coords[1].split(',')))
        brick = Brick(id, start, end)
        world.add_brick(brick)
    return world


# Main Function
def main(input_lines):
    world = parse_input(input_lines)
    world.settle_bricks()
    return world.simulate_disintegration()


# Read the input file
input_file_path = "input.txt"  # Ensure this path is correct for your environment

with open(input_file_path, "r") as file:
    input_lines = file.readlines()

# Running the function with the input file content and getting the result
safe_disintegrations = main(input_lines)
print(f"Number of bricks that can be safely disintegrated: {safe_disintegrations}")


ValueError: Brick 211 is not stable!

In [4]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.start = tuple(min(a, b) for a, b in zip(coord1, coord2))
        self.end = tuple(max(a, b) for a, b in zip(coord1, coord2))
        self.supported = False  # Initially, no brick is supported

    def update_support(self, brick_map):
        # A brick is supported if any part of it is directly above another brick or the ground
        if self.start[2] == 1:  # It's on the ground
            self.supported = True
            return

        for x in range(self.start[0], self.end[0] + 1):
            for y in range(self.start[1], self.end[1] + 1):
                if (x, y, self.start[2] - 1) in brick_map:
                    self.supported = True
                    return  # As soon as we find support, no need to check further

        self.supported = False  # If no part is supported, the brick is not supported

class World:
    def __init__(self):
        self.bricks = []
        self.brick_map = {}  # Mapping of coordinates to bricks

    def add_brick(self, brick):
        self.bricks.append(brick)
        # Add all coordinates occupied by this brick to the brick map
        for x in range(brick.start[0], brick.end[0] + 1):
            for y in range(brick.start[1], brick.end[1] + 1):
                for z in range(brick.start[2], brick.end[2] + 1):
                    self.brick_map[(x, y, z)] = brick

    def settle_bricks(self):
        # Update support status for all bricks
        for brick in self.bricks:
            brick.update_support(self.brick_map)

        # Check if any brick is unsupported
        for brick in self.bricks:
            if not brick.supported:
                raise ValueError(f"Brick {brick.id} is not stable!")

    def simulate_disintegration(self):
        safe_disintegrations = 0
        for brick in self.bricks:
            # Temporarily remove the brick
            for x in range(brick.start[0], brick.end[0] + 1):
                for y in range(brick.start[1], brick.end[1] + 1):
                    for z in range(brick.start[2], brick.end[2] + 1):
                        del self.brick_map[(x, y, z)]

            # Check stability of all other bricks
            all_stable = True
            for other_brick in self.bricks:
                if other_brick is not brick:
                    other_brick.update_support(self.brick_map)
                    if not other_brick.supported:
                        all_stable = False
                        break

            # If all other bricks are stable, this disintegration is safe
            if all_stable:
                safe_disintegrations += 1

            # Put the brick back
            self.add_brick(brick)

            # Update all bricks support status after adding back
            for other_brick in self.bricks:
                other_brick.update_support(self.brick_map)

        return safe_disintegrations


def parse_input(input_lines):
    world = World()
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        start = tuple(map(int, coords[0].split(',')))
        end = tuple(map(int, coords[1].split(',')))
        brick = Brick(id, start, end)
        world.add_brick(brick)
    return world


# Main Function
def main(input_lines):
    world = parse_input(input_lines)
    world.settle_bricks()
    return world.simulate_disintegration()


# Read the input file
input_file_path = "input.txt"  # Path to the uploaded file

with open(input_file_path, "r") as file:
    input_lines = file.readlines()

# Running the function with the input file content and getting the result
safe_disintegrations = main(input_lines)
safe_disintegrations


ValueError: Brick 1 is not stable!

In [6]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.coords = [coord1, coord2]  # Two coordinates defining the brick
        self.supported = False  # Initially, no brick is assumed to be supported

    def is_stable(self, bricks):
        # Check if the brick is on the ground or supported by another brick
        if any(coord[2] == 1 for coord in self.coords):  # On the ground
            return True
        for other in bricks:
            if other is not self and self.is_supported_by(other):
                return True
        return False

    def is_supported_by(self, other):
        # Check if 'other' brick supports this brick
        x_range = range(min(self.coords[0][0], self.coords[1][0]), max(self.coords[0][0], self.coords[1][0]) + 1)
        y_range = range(min(self.coords[0][1], self.coords[1][1]), max(self.coords[0][1], self.coords[1][1]) + 1)
        for x in x_range:
            for y in y_range:
                if (x, y, self.lowest_z() - 1) in other.all_coords():
                    return True
        return False

    def lowest_z(self):
        return min(self.coords[0][2], self.coords[1][2])

    def all_coords(self):
        # Generate all coordinates occupied by this brick
        for x in range(min(self.coords[0][0], self.coords[1][0]), max(self.coords[0][0], self.coords[1][0]) + 1):
            for y in range(min(self.coords[0][1], self.coords[1][1]), max(self.coords[0][1], self.coords[1][1]) + 1):
                for z in range(min(self.coords[0][2], self.coords[1][2]), max(self.coords[0][2], self.coords[1][2]) + 1):
                    yield (x, y, z)

def parse_input(input_lines):
    bricks = []
    for id, line in enumerate(input_lines):
        parts = line.strip().split('~')
        coord1 = tuple(map(int, parts[0].split(',')))
        coord2 = tuple(map(int, parts[1].split(',')))
        bricks.append(Brick(id, coord1, coord2))
    return bricks

def simulate_disintegration(bricks):
    safe_disintegrations = 0
    for brick in bricks:
        if brick.is_stable(bricks):  # Check if the brick is initially stable
            # Temporarily remove the brick and check stability of other bricks
            bricks.remove(brick)
            if all(other.is_stable(bricks) for other in bricks):
                safe_disintegrations += 1  # If all bricks are still stable, this disintegration is safe
            bricks.append(brick)  # Put the brick back for the next iteration
    return safe_disintegrations

# Read the input file
input_file_path = "input.txt"  # Path to the uploaded file

with open(input_file_path, "r") as file:
    input_lines = file.readlines()

# Parse the input and run the simulation
bricks = parse_input(input_lines)
safe_disintegrations = simulate_disintegration(bricks)
safe_disintegrations


0

In [7]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.coord1 = coord1
        self.coord2 = coord2
        self.supported = False

    def get_volume(self):
        return (abs(self.coord2[0] - self.coord1[0]) + 1) * \
               (abs(self.coord2[1] - self.coord1[1]) + 1) * \
               (abs(self.coord2[2] - self.coord1[2]) + 1)

    def is_supported(self, bricks, ground_level=1):
        if min(self.coord1[2], self.coord2[2]) == ground_level:
            return True  # The brick is on the ground
        # Check for support from other bricks
        for brick in bricks:
            if brick == self:
                continue
            if self.is_overlapping_below(brick):
                return True
        return False

    def is_overlapping_below(self, other):
        # Check if 'other' brick is directly under this one
        x_overlap = self.coord1[0] <= other.coord2[0] and self.coord2[0] >= other.coord1[0]
        y_overlap = self.coord1[1] <= other.coord2[1] and self.coord2[1] >= other.coord1[1]
        z_overlap = self.coord1[2] == other.coord2[2] + 1 or self.coord2[2] == other.coord1[2] - 1
        return x_overlap and y_overlap and z_overlap

def parse_input(input_lines):
    bricks = []
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        coord1 = tuple(map(int, coords[0].split(',')))
        coord2 = tuple(map(int, coords[1].split(',')))
        bricks.append(Brick(id, coord1, coord2))
    return bricks

def simulate_disintegration(bricks):
    safe_disintegrations = 0
    for brick in bricks:
        bricks_temp = bricks.copy()
        bricks_temp.remove(brick)
        if all(b.is_supported(bricks_temp) for b in bricks_temp):
            safe_disintegrations += 1
    return safe_disintegrations

# Read and process the input file
input_file_path = "input.txt"  # Update this to the correct path

with open(input_file_path, 'r') as file:
    input_lines = file.readlines()

bricks = parse_input(input_lines)
safe_disintegrations = simulate_disintegration(bricks)
print(safe_disintegrations)


0


In [8]:
class Brick:
    def __init__(self, id, start, end):
        self.id = id
        self.start = start
        self.end = end
        self.supports = set()  # Other bricks that this brick supports
        self.supported_by = set()  # Other bricks that support this brick

    def overlaps(self, other):
        # Check if this brick overlaps with 'other' in x and y, and is directly above in z
        x_overlap = not (self.end[0] < other.start[0] or self.start[0] > other.end[0])
        y_overlap = not (self.end[1] < other.start[1] or self.start[1] > other.end[1])
        z_above = self.start[2] == other.end[2] + 1
        return x_overlap and y_overlap and z_above

def parse_input(input_lines):
    bricks = []
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        start = tuple(map(int, coords[0].split(',')))
        end = tuple(map(int, coords[1].split(',')))
        bricks.append(Brick(id, start, end))
    return bricks

def establish_supports(bricks):
    for brick in bricks:
        for other in bricks:
            if brick.overlaps(other):
                brick.supported_by.add(other)
                other.supports.add(brick)

def safe_disintegrate_count(bricks):
    safe_count = 0
    for brick in bricks:
        # A brick is safe to remove if every brick it supports is also supported by others
        if all(support != brick and support.supported_by for support in brick.supports):
            safe_count += 1
    return safe_count

# Reading and processing the input
input_file_path = "input.txt"  # Update this path to your input file location
with open(input_file_path, 'r') as file:
    input_lines = file.readlines()

bricks = parse_input(input_lines)
establish_supports(bricks)
safe_count = safe_disintegrate_count(bricks)
print(safe_count)


1249


In [9]:
class Brick:
    def __init__(self, id, coord1, coord2):
        self.id = id
        self.min_x = min(coord1[0], coord2[0])
        self.max_x = max(coord1[0], coord2[0])
        self.min_y = min(coord1[1], coord2[1])
        self.max_y = max(coord1[1], coord2[1])
        self.min_z = min(coord1[2], coord2[2])
        self.max_z = max(coord1[2], coord2[2])
        self.supported_by = set()

def parse_input(input_lines):
    bricks = []
    for id, line in enumerate(input_lines):
        coords = line.strip().split('~')
        coord1 = tuple(map(int, coords[0].split(',')))
        coord2 = tuple(map(int, coords[1].split(',')))
        bricks.append(Brick(id, coord1, coord2))
    return bricks

def establish_supports(bricks):
    for i, brick in enumerate(bricks):
        for j, other in enumerate(bricks):
            if i != j:
                # Check if 'other' is directly below 'brick'
                if other.max_z == brick.min_z - 1 and \
                    other.min_x <= brick.max_x and other.max_x >= brick.min_x and \
                    other.min_y <= brick.max_y and other.max_y >= brick.min_y:
                    brick.supported_by.add(other)

def find_safe_to_remove(bricks):
    safe_to_remove = 0
    for brick in bricks:
        # A brick is safe to remove if all the bricks it's supporting are also supported by others
        if all(len(b.supported_by - {brick}) > 0 for b in bricks if brick in b.supported_by):
            safe_to_remove += 1
    return safe_to_remove

# Read the input file and process
input_file_path = "input.txt"  # Make sure this path is correct
with open(input_file_path, 'r') as file:
    input_lines = file.readlines()

# Parse the input, establish supports, and find the number of bricks safe to remove
bricks = parse_input(input_lines)
establish_supports(bricks)
safe_count = find_safe_to_remove(bricks)
print(safe_count)


896


In [10]:
import itertools

class Brick:
    def __init__(self, coord1, coord2):
        self.coord1 = tuple(min(a, b) for a, b in zip(coord1, coord2))
        self.coord2 = tuple(max(a, b) for a, b in zip(coord1, coord2))
        # Assuming bricks are stable initially
        self.stable = True

    def supports(self, other):
        # Checks if this brick supports the other brick
        x_overlap = self.coord1[0] <= other.coord2[0] and self.coord2[0] >= other.coord1[0]
        y_overlap = self.coord1[1] <= other.coord2[1] and self.coord2[1] >= other.coord1[1]
        z_support = self.coord1[2] == other.coord2[2] + 1
        return x_overlap and y_overlap and z_support

def read_bricks(input_lines):
    bricks = []
    for line in input_lines:
        parts = line.strip().split('~')
        coord1 = tuple(map(int, parts[0].split(',')))
        coord2 = tuple(map(int, parts[1].split(',')))
        bricks.append(Brick(coord1, coord2))
    return bricks

def find_stable_bricks(bricks):
    # Check each pair of bricks to establish support relationships
    for a, b in itertools.combinations(bricks, 2):
        # If a supports b or b supports a, both are considered stable
        if a.supports(b) or b.supports(a):
            a.stable = b.stable = True

def safe_to_remove(bricks):
    # Count the bricks that, if removed, do not affect the stability of others
    find_stable_bricks(bricks)
    return sum(not brick.stable or all(not other.stable or not brick.supports(other) for other in bricks) for brick in bricks)

# Read the input file and process
with open("input.txt", 'r') as file:  # Ensure the file path is correct
    input_lines = file.readlines()

bricks = read_bricks(input_lines)
print(safe_to_remove(bricks))


833


In [2]:
with open('input.txt', 'r') as file:
    inp = file.read()

In [3]:

from collections import defaultdict

brick = []
for line in inp.split("\n"):
    a,b = line.split("~")
    a = list(map(int, a.split(",")))
    b = list(map(int, b.split(",")))
    brick.append((a,b))

n = len(brick)

brick.sort(key=lambda x: x[0][2])

highest = defaultdict(lambda:(0,-1))
bad = set()
graph = [[] for i in range(n)]
for idx,b in enumerate(brick):
    mxh = -1
    support_set = set()
    for x in range(b[0][0], b[1][0]+1):
        for y in range(b[0][1], b[1][1]+1):
            if highest[x,y][0] + 1 > mxh:
                mxh = highest[x,y][0] + 1
                support_set = {highest[x,y][1]}
            elif highest[x,y][0] + 1 == mxh:
                support_set.add(highest[x,y][1])
    
    for x in support_set:
        if x != -1:
            graph[x].append(idx)

    if len(support_set) == 1:
        bad.add(support_set.pop())
    
    fall = b[0][2] - mxh
    if fall > 0:
        b[0][2] -= fall
        b[1][2] -= fall

    for x in range(b[0][0], b[1][0]+1):
        for y in range(b[0][1], b[1][1]+1):
            highest[x,y] = (b[1][2], idx)

print(len(brick)-len(bad)-1) # -1 for "ground" brick

def count(idx, graph):
    indeg = [0 for __ in range(n)]
    for j in range(n):
        for i in graph[j]:
            indeg[i] += 1
    q = [idx]
    count = -1
    while len(q) > 0:
        count += 1
        x = q.pop()
        for i in graph[x]:
            indeg[i] -= 1
            if indeg[i] == 0:
                q.append(i)

    return count

print(sum(count(x, graph) for x in range(n)))


ValueError: not enough values to unpack (expected 2, got 1)

In [4]:
# Given code to solve the brick problem
from collections import defaultdict

# Assume inp is the input string from the input file
with open("input.txt", 'r') as file:
    inp = file.read()

brick = []
for line in inp.split("\n"):
    if line.strip():  # Ensure the line has content
        a, b = line.split("~")
        a = list(map(int, a.split(",")))
        b = list(map(int, b.split(",")))
        brick.append((a, b))

n = len(brick)
brick.sort(key=lambda x: x[0][2])

highest = defaultdict(lambda: (0, -1))
bad = set()
graph = [[] for i in range(n)]
for idx, b in enumerate(brick):
    mxh = -1
    support_set = set()
    for x in range(b[0][0], b[1][0] + 1):
        for y in range(b[0][1], b[1][1] + 1):
            if highest[x, y][0] + 1 > mxh:
                mxh = highest[x, y][0] + 1
                support_set = {highest[x, y][1]}
            elif highest[x, y][0] + 1 == mxh:
                support_set.add(highest[x, y][1])

    for x in support_set:
        if x != -1:
            graph[x].append(idx)

    if len(support_set) == 1:
        bad.add(support_set.pop())

    fall = b[0][2] - mxh
    if fall > 0:
        b[0][2] -= fall
        b[1][2] -= fall

    for x in range(b[0][0], b[1][0] + 1):
        for y in range(b[0][1], b[1][1] + 1):
            highest[x, y] = (b[1][2], idx)

def count(idx, graph):
    indeg = [0 for __ in range(n)]
    for j in range(n):
        for i in graph[j]:
            indeg[i] += 1
    q = [idx]
    cnt = -1
    while len(q) > 0:
        cnt += 1
        x = q.pop()
        for i in graph[x]:
            indeg[i] -= 1
            if indeg[i] == 0:
                q.append(i)

    return cnt

first_result = len(brick) - len(bad) - 1  # -1 for "ground" brick
second_result = sum(count(x, graph) for x in range(n))

first_result, second_result


(430, 63166)

In [5]:
from aoc_tools import *
import sys
sys.setrecursionlimit(1000000)
ans = res = 0

with open("input.txt") as f:
    s = f.read().strip()


bricks = []
# line by line input
for l in s.split("\n"):
    sloc,eloc = l.split("~")
    sx,sy,sz = map(int,sloc.split(","))
    ex,ey,ez = map(int,eloc.split(","))
##    sx,ex = min(sx,ex),max(sx,ex)
##    sy,ey = min(sy,ey),max(sy,ey)
##    sz,ez = min(sz,ez),max(sz,ez)
    bricks.append((sx,sy,sz,ex,ey,ez))

# z is vertical direction
# z = 0

def is_solid(bset, x,y,z):
    if z == 0:
        return True
    return (x,y,z) in bset
##    for (sx,sy,sz,ex,ey,ez) in bricks:
##        if x in range(sx,ex+1) and y in range(sy,ey+1) and z in range(sz,ez+1):
##            return True
    ##return False

def bfall(bricks):
    fell = False
    bset = set()
    for (sx,sy,sz,ex,ey,ez) in bricks:
        for x in range(sx,ex+1):
            for y in range(sy,ey+1):
                bset.add((x,y,ez))
    
    new_bricks = []
    for b in bricks:
        supp = False
        sx,sy,sz,ex,ey,ez = b
        for x in range(sx,ex+1):
            for y in range(sy,ey+1):
                # is (x,y,sz-1) solid?
                if is_solid(bset, x, y, sz - 1):
                    supp = True
                    break
            if supp:
                break
        if not supp:
            fell = True
            new_bricks.append((sx,sy,sz-1, ex,ey,ez-1))
        else:
            new_bricks.append(b)
    return fell, new_bricks


fell = True
cnt = 0
while fell:
    cnt += 1
    fell, bricks = bfall(bricks)

bcopy = bricks.copy()

ans = 0
for i in range(len(bcopy)):
    bcopy2 = bcopy.copy()
    del bcopy2[i]
    if not bfall(bcopy2)[0]:
        ans += 1
print_(ans)

ans = 0
for i in range(len(bcopy)):
    bcopy2 = bcopy.copy()
    del bcopy2[i]
    bcopy3 = bcopy2.copy()
    fell = True
    while fell:
        fell, bcopy2 = bfall(bcopy2)
    mm = 0
    for a,b in zip(bcopy2, bcopy3):
        if a != b:
            mm += 1
    ans += mm
print_(ans)

ModuleNotFoundError: No module named 'aoc_tools'

In [6]:
def whatif(disintegrated):
	falling = set()
	def falls(brick):
		if brick in falling:
			return
		falling.add(brick)
		for parent in above[brick]:
			if not len(below[parent] - falling):
				# if everything below the parent is falling, so is the parent
				falls(parent)
	falls(disintegrated)
	return len(falling)

p1 = 0
p2 = 0
for brick in fallen:
	wouldfall = whatif(brick)
	p1 += wouldfall == 1
	p2 += wouldfall - 1

print(p1, p2) 

NameError: name 'fallen' is not defined

In [8]:
import re
from collections import defaultdict
def ints(s):
	return list(map(int, re.findall(r'\d+', s)))

ll = [tuple(ints(x) + [i]) for i, x in enumerate(open('input.txt').read().strip().split('\n'))]

def down(brick):
	return (brick[0], brick[1], brick[2] - 1, brick[3], brick[4], brick[5] - 1, brick[6])

def positions(brick):
	for x in range(brick[0], brick[3] + 1):
		for y in range(brick[1], brick[4] + 1):
			for z in range(brick[2], brick[5] + 1):
				yield (x,y,z)

occupied = {}
fallen = []
for brick in sorted(ll, key=lambda brick: brick[2]):
	while True:
		nxt = down(brick)
		if not any(pos in occupied for pos in positions(nxt)) and nxt[2] > 0:
			brick = nxt
		else:
			for pos in positions(brick):
				occupied[pos] = brick
			fallen.append(brick)
			break

above = defaultdict(set)
below = defaultdict(set)
for brick in fallen:
	inthisbrick = set(positions(brick))
	for pos in positions(down(brick)):
		if pos in occupied and pos not in inthisbrick:
			above[occupied[pos]].add(brick)
			below[brick].add(occupied[pos])


def whatif(disintegrated):
	falling = set()
	def falls(brick):
		if brick in falling:
			return
		falling.add(brick)
		for parent in above[brick]:
			if not len(below[parent] - falling):
				# if everything below the parent is falling, so is the parent
				falls(parent)
	falls(disintegrated)
	return len(falling)

p1 = 0
p2 = 0
for brick in fallen:
	wouldfall = whatif(brick)
	p1 += wouldfall == 1
	p2 += wouldfall - 1

print(p1, p2)


432 63166
